<a href="https://colab.research.google.com/github/hianaracamilla/controle-guias/blob/main/Controle_de_guias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTANDO O PANDAS

In [1]:
import pandas as pd

IMPORTANDO A PÁGINA HTML E VERIFICANDO QUANTAS TABELAS HÁ

In [18]:
viagens = pd.read_html('/content/tours_atual.html')
len(viagens)

6

SELECIONANDO A TABELA COM OS DADOS EM UM DATAFRAME

In [19]:
df_viagens = viagens[3]

Apagando linhas e colunas que não serão usadas e resetando o index

In [20]:
df_viagens.drop([0,1,2,3,4], axis=0, inplace=True)
df_viagens.drop([1,3,4,5, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18], axis=1, inplace=True)
df_viagens.reset_index(drop=True, inplace=True, )

Transformando a primeira linha em cabeçalho

In [21]:
df_viagens.columns = df_viagens.iloc[0]
df_viagens = df_viagens[1:]
df_viagens.reset_index(drop=True, inplace=True, )

Criando cópia de segurança, retirando NA da coluna Guia, somando a quantidade de passageiros, e agrupando por guia

In [22]:
df = df_viagens.copy()

df = df[df['Guia'].notna()]

df.loc[:, 'PAX'] = df['PAX'].apply(lambda x: eval(x.replace('+', '+')) if '+' in x else int(x))

df.loc[:, 'Total PAX por Guia'] = df.groupby('Guia')['PAX'].transform('sum')

Transformando a data, para deixar apenas os numeros

In [23]:
df.loc[:, 'Data'] = df['Data'].str.split(' ').str[1]

Apagando duplicados com 2 critérios

In [24]:
df = df.drop_duplicates(subset=['Guia', 'Tour'])

Reorganizando as colunas

In [25]:
df = df[['Data', 'Guia', 'Total PAX por Guia', 'Tour']]

Filtrando tours não inclusos

In [26]:
patterns_to_remove = ['Concha Y Toro', 'Alyan', 'Privativo', 'Cousiño', 'Transporte', 'Undurraga', 'Sunset El Principal', 'Parque Safari']
df = df[~df['Tour'].str.startswith(tuple(patterns_to_remove))]
df.reset_index(drop=True, inplace=True, )

Função para ajustar nome do tour com critérios

In [27]:
def ajustar_nome_tour(row):
    if row['Total PAX por Guia'] > 19:
        tour_name = row['Tour']
        if tour_name.startswith('City Tour'):
            return tour_name  # Pula City Tour
        elif tour_name == 'Portillo E Laguna Del Inca':
            return 'Portillo Bus'
        elif tour_name == 'Farellones Full Day':
            return 'Farellones Full Day BUS'
        elif tour_name == 'Valparaiso E Vina Del Mar':
            return 'Valparaiso BUS'
        else:
            return tour_name + ' BUS'
    else:
        return row['Tour']

Aplicando função na coluna tour.

In [28]:
df['Tour'] = df.apply(ajustar_nome_tour, axis=1)
df = df.sort_values(by='Tour')

Exportando como excel

In [29]:
df.to_excel('guia_x_tour.xlsx', index=False)